In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

In [2]:
# lineare klassifikation

def e_in(X, y, w):
    return np.sum(H(X, w) != y) / X.shape[0]

# perzeptron (x, w -> y)
def h(x, w):
    return np.sign(w @ x)

def H(X, w):
    return [h(x, w) for x in X]

# p(x) (x, w -> gerade)
def p(x, w):
    return -w[1]/w[2]*x - w[0]/w[2]

def P(X, w):
    return [p(x, w) for x in X]

# pla (X, y -> w)
def pla(X, y, w=None, t=0):
    if t == 0:
        X = np.column_stack((np.ones(X.shape[0]), X))
    if w is None:
        w = np.zeros(X.shape[1])

    for i, x in enumerate(X):
        if h(x, w) != y[i]:
            return pla(X, y, w + y[i]*x, t+1)

    return w

# pocket (X, y -> w)
def pocket(X, y, T):
    X = np.column_stack((np.ones(X.shape[0]), X))

    w = np.zeros(X.shape[1])
    w_d = w.copy()
    e = 1

    for _ in range(T):
        for i, x in enumerate(X):
            if h(x, w) != y[i]:
                w += y[i]*x

                if e_in(X, y, w) < e:
                    w_d = w.copy()
                    e = e_in(X, y, w_d)

    return w_d

def scatter(X, y, w):
    xlim = (X[:,0].min()-0.1, X[:,0].max()+0.1)
    ylim = (X[:,1].min()-0.1, X[:,1].max()+0.1)

    plt.scatter(X[:,0], X[:,1], c=y)
    plt.plot(xlim, P(xlim, w))
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()

In [1]:
# lineare regression

def e_in(X, y, w, Q=4):
    g = G(X, w, Q)

    return float((g-y).T@(g-y) / X.shape[0])

def e_out(f, w, start=-1, stop=1, K=50):
    L = np.linspace(start, stop, K)
    
    return e_in(L, f(L), w)
    
# g ~ f (x, w -> y)
def g(x, w):
    return np.hstack((np.ones(1), x)) @ w

def G(X, w):
    return [g(x, w) for x in X]

# lineare regression (X, y -> w)
def lin_reg(X, y):
    X = np.column_stack((np.ones(X.shape[0]), X))

    X_d = np.linalg.inv(X.T @ X) @ X.T
    return X_d @ y

def scatter(X, y, w):
    xlim = (X.min()-0.1, X.max()+0.1)
    ylim = (y.min()-0.1, y.max()+0.1)

    plt.xlabel("X")
    plt.ylabel("y")

    plt.legend(handles=[
        mpatches.Patch(color="#1f77b4", label="f(x)"),
        mpatches.Patch(color="#ff7f0e", label="g(x)")
    ])

    plt.scatter(X, y, c="#1f77b4")
    plt.plot(xlim, G(xlim, w), c="#ff7f0e")
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.show()

In [ ]:
# lineare regression mit featuretransformation

# featuretranformation (X -> Z)
def phi(x, Q=4):
    return np.array([x**i for i in range(1, Q+1)])

def PHI(X, Q=4):
    return np.concatenate([phi(x, Q).reshape(1, -1) for x in X])

# g ~ f (x -> z; z, w -> y)
def g(x, w, Q=4):
    return np.hstack((np.ones(1), phi(x, Q))) @ w

def G(X, w, Q=4):
    return [g(x, w, Q) for x in X]

# lineare regresseion (X -> Z; Z, y -> w)
def lin_reg_featuretransformed(X, y, Q=4):
    return lin_reg(PHI(X, Q), y)
    
def scatter(X, y, w, f=None, Q=4, start=None, stop=None, K=50):
    if start is None or stop is None:
        L = np.linspace(X.min(), X.max(), K)
    else:
        L = np.linspace(start, stop, K)

    plt.xlabel("X")
    plt.ylabel("y")

    plt.legend(handles=[
        mpatches.Patch(color="#1f77b4", label="f(x)"),
        mpatches.Patch(color="#ff7f0e", label="g(x)")
    ])

    plt.scatter(X, y, c="#1f77b4")
    if f is not None: plt.plot(L, f(L), c="#1f77b4")
    plt.plot(L, G(L, w, Q), c="#ff7f0e")
    plt.show()

In [ ]:
# lineare regression mit regularisierung

# lineare regression (X, y -> w)
def lin_reg_regularized(X, y, l=0):
    X = np.column_stack((np.ones(X.shape[0]), X))

    X_d = np.linalg.inv(X.T @ X + l * np.identity(np.min(X.shape))) @ X.T
    return X_d @ y

In [ ]:
# lineare regression mit featuretransformation und regularisierung

# lineare regression (X -> Z; Z, y -> w)
def lin_reg_featuretransformed_regularized(X, y, Q=4, l=0):
    return lin_reg_regularized(PHI(X, Q), l)